In [3]:
from sklearn.datasets import load_iris,load_boston
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import KernelPCA
#from sklearn.metrics import normalized_mutual_info_score as nMI
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import scipy.spatial as ss
from scipy.special import digamma,gamma
from math import log,pi
import numpy.random as nr
import random

In [24]:
##old version
def featureSelection(alist):
    b=sorted(alist)
    #print b
    index=[]
    for i in b:
        if i in alist:
            index.append(alist.index(i))
    return index

In [27]:
def InxOfSortedFea(feaInfo):
    """return sorted index of a list"""
    sortInfo=sorted(feaInfo,reverse=True)
    ##for test
    ##print sortInfo
    return [feaInfo.index(i) for i in sortInfo]
#module test
#a=[3,5,1,2,7]
#print InxOfSortedFea(a)

In [15]:
def corDis(x,y):
    assert len(x)==len(y)
    n=len(x)
    a=np.zeros((n,n))
    b=np.zeros((n,n))
    for i in xrange(n):
        for j in xrange(n):
            a[i,j]=np.linalg.norm(x[i]-x[j])
            b[i,j]=np.linalg.norm(y[i]-y[j])
    aRowMean=np.mean(a,axis=1)
    aColMean=np.mean(a,axis=0)
    bRowMean=np.mean(b,axis=1)
    bColMean=np.mean(b,axis=0)
    aMean=np.mean(a)
    bMean=np.mean(b)
    A=np.zeros((n,n))
    B=np.zeros((n,n))
    for i in xrange(n):
        for j in xrange(n):
            A[i,j]=a[i,j]-aRowMean[i]-aColMean[j]+aMean
            B[i,j]=b[i,j]-bRowMean[i]-bColMean[j]+bMean
    cordis=0.
    for i in xrange(n):
        for j in xrange(n):
            cordis += A[i,j]*B[i,j]
    return cordis*(1./n**2)

In [151]:
import numpy as np
a=np.random.randint(0,20,size=(4,3))
print a


[[ 7 14  3]
 [ 1  8  8]
 [16 11 19]
 [ 0  2 19]]


In [157]:
print np.mean(a,axis=0)
print a-np.mean(a,axis=0)
print np.mean(a,axis=1)
#print a-np.mean(a,axis=1)
print np.mean(a)

[  6.     8.75  12.25]
[[  1.     5.25  -9.25]
 [ -5.    -0.75  -4.25]
 [ 10.     2.25   6.75]
 [ -6.    -6.75   6.75]]
[  8.           5.66666667  15.33333333   7.        ]
9.0


In [165]:
n=8
n**2

64

In [63]:
def MSE(target,predictions):
    SSE=np.power(target-predictions,2)
    return np.mean(SSE)

In [40]:
def KthFold(x,ith,kfold=5):
    n=len(x)
    
    test=x[(0+ith)*n/kfold:(1+ith)*n/kfold]
    train=np.delete(x,test)
    return test,train
##module test
#a=range(13)
#test,train=KthFold(x=a,ith=1,kfold=3)
#print test,train

In [72]:
def cross_validation(data,y,index):
    col=[]
    kfold=range(data.shape[0])
    random.shuffle(kfold)
    ##five fold observation
    k=5
    for i in range(len(index)):
        indexChoiceError=[]
        for j in xrange(k):
            lr=LinearRegression(fit_intercept=True)
            ##kfold cross validation
            test,train=KthFold(x=kfold,ith=j)
            ##can not slicing data on row and col at same time,so slicing two step
            stepData=data[:,index[:i+1]]
            #print stepData.shape
            lr.fit(stepData[train],[y[n] for n in train])
            indexChoiceError.append(MSE([y[n] for n in test],lr.predict(stepData[test])))
        col.append(np.mean(indexChoiceError))
    return col

In [81]:
def layer(x,y):
    kpca=KernelPCA(kernel='rbf',n_components=x.shape[1])
    xT=kpca.fit_transform(x)
    rank=[]
    features=xT.shape[1]
    #print xT[:,0]
    #print y[:]
    for i in range(features):
        rank.append(corDis(xT[:,i],y))
    #select features
    feaSelected=InxOfSortedFea(rank)
    #lr = LinearRegression(fit_intercept=True)
    #return feaSelected
    crossError=cross_validation(xT,y,feaSelected)
    #plt.scatter(x=range(feaSelected),y=cross_error)
    ##return selected error
    return xT[:,:crossError.index(min(crossError))]
#print layer(x,y)    

In [7]:
boston=load_boston()
c=[2,4,5,6,7,12]
y=boston.target
x=boston.data[:,c]
print x[:5],y[:5]

[[  2.31     0.538    6.575   65.2      4.09     4.98  ]
 [  7.07     0.469    6.421   78.9      4.9671   9.14  ]
 [  7.07     0.469    7.185   61.1      4.9671   4.03  ]
 [  2.18     0.458    6.998   45.8      6.0622   2.94  ]
 [  2.18     0.458    7.147   54.2      6.0622   5.33  ]] [ 24.   21.6  34.7  33.4  36.2]


In [9]:
#debug
kpca=KernelPCA(kernel='rbf',n_components=x.shape[1])
xT=kpca.fit_transform(x)
print xT[:5],xT.shape

[[ 0.03712085  0.01360258  0.02278033  0.0077594   0.01296009 -0.02171701]
 [ 0.03949152  0.01481318  0.02533058  0.0089317   0.01515028 -0.02615883]
 [ 0.03308646  0.0116108   0.01871436  0.0059902   0.00973848 -0.0155123 ]
 [ 0.03344029  0.01177521  0.01903256  0.00611776  0.00996354 -0.01592664]
 [ 0.03518263  0.01266409  0.02090234  0.00697378  0.01155848 -0.01913772]] (506L, 6L)


In [18]:
#debug
rank=[]
features=xT.shape[1]
print features
for i in range(features):
        rank.append(corDis(xT[:,i],y))

6


In [28]:
#debug
print rank
feaSelected=InxOfSortedFea(rank)
print feaSelected

[0.030737471997507353, 0.028751781048373771, 0.025728080395453604, 0.020441153389522654, 0.018887039843905825, 0.016849266004061145]
[0, 1, 2, 3, 4, 5]


In [39]:
#debug
#crossError=cross_validation(xT,y,feaSelected)
data,y,index=xT,y,feaSelected
#print data[:5],y[:5],index
#print xT[:5],y[:5],feaSelected
col=[]
kfold=range(data.shape[0])
random.shuffle(kfold)
print col,len(kfold)


[] 506


In [60]:
#debug
i=1
j=0
indexChoiceError=[]
lr=LinearRegression(fit_intercept=True)
test,train=KthFold(x=kfold,ith=j)
print len(test),len(train)
stepData=data[:,index[:i]]
print stepData[:5]

print index[:i]
print index[:0]
print data[:5,0]

101 405
[[ 0.03712085]
 [ 0.03949152]
 [ 0.03308646]
 [ 0.03344029]
 [ 0.03518263]]
[0]
[]
[ 0.03712085  0.03949152  0.03308646  0.03344029  0.03518263]


In [70]:
#debug
lr.fit(stepData[train],[y[n] for n in train])
print stepData[test].shape
indexChoiceError.append(MSE([y[n] for n in test],lr.predict(stepData[test])))
print indexChoiceError

(101L, 1L)
[66.159182388375484]


In [ ]:
#debug

In [ ]:
#debug

In [ ]:
#debug

'asdfa'

In [82]:
result1=layer(x,y)
print result1[:5]


[[-0.03712085 -0.01360258 -0.02278033  0.0077594   0.01296009]
 [-0.03949152 -0.01481318 -0.02533058  0.0089317   0.01515028]
 [-0.03308646 -0.0116108  -0.01871436  0.0059902   0.00973848]
 [-0.03344029 -0.01177521 -0.01903256  0.00611776  0.00996354]
 [-0.03518263 -0.01266409 -0.02090234  0.00697378  0.01155848]]


In [84]:
print result1.shape
print x.shape

(506L, 5L)
(506L, 6L)


In [144]:
def mi(x,y,k=3,base=2):
  """ Mutual information of x and y
      x,y should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
      if x is a one-dimensional scalar and we have four samples
  """
  #print x
  #print y  
  assert len(x)==len(y), "Lists should have same length"
  assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
  intens = 1e-10 #small noise to break degeneracy, see doc.
  x = [list(p + intens*nr.rand(len(x[0]))) for p in x]
  y = [list(p + intens*nr.rand(len(y[0]))) for p in y]
  points = zip2(x,y)
  #Find nearest neighbors in joint space, p=inf means max-norm
  tree = ss.cKDTree(points)
  dvec = [tree.query(point,k+1,p=float('inf'))[0][k] for point in points]
  a,b,c,d = avgdigamma(x,dvec), avgdigamma(y,dvec), digamma(k), digamma(len(x)) 
  return (-a-b+c+d)/log(base)

In [21]:
test=load_boston()
#?test
y=test.target
data=test.data
import matplotlib.pyplot as plt
f,ax=plt.subplots(ncols=3,nrows=5)
ax[0,0].scatter(x=data[:,0], y=y)

ax[0,1].scatter(x=data[:,1], y=y)

ax[0,2].scatter(x=data[:,2], y=y)

ax[1,0].scatter(x=data[:,3], y=y)

ax[1,1].scatter(x=data[:,4], y=y)

ax[1,2].scatter(x=data[:,5], y=y)

ax[2,0].scatter(x=data[:,6], y=y)

ax[2,1].scatter(x=data[:,7], y=y)

ax[2,2].scatter(x=data[:,8], y=y)

ax[3,0].scatter(x=data[:,9], y=y)

ax[3,1].scatter(x=data[:,10], y=y)

ax[3,2].scatter(x=data[:,11], y=y)

ax[4,0].scatter(x=data[:,12], y=y)


plt.show()

In [42]:
import random
f=range(20)
print random.shuffle(f)
print f

None
[3, 8, 9, 1, 0, 16, 12, 17, 18, 4, 11, 15, 5, 14, 2, 13, 10, 19, 7, 6]


In [36]:
f=range(20)
print f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [63]:
for i in xrange(5):
    print i

0
1
2
3
4
